# Śledzenie obiektów

<img src="https://i.imgur.com/wKXXFkQ.png" width="500">

## Wstęp
W erze cyfrowej, w obliczu rosnącej lawinowo ilości danych wideo, zdolność do ich automatycznego rozpoznawania i interpretowania staje się kluczowa w wielu dziedzinach – od bezpieczeństwa publicznego po autonomiczne pojazdy. Technologie oparte na głębokim uczeniu rewolucjonizują sposób, w jaki przetwarzamy informacje wizualne. Kluczowym wyzwaniem jest tu detekcja i śledzenie obiektów na filmach wideo.

Celem tego zadania jest opracowanie algorytmu, który będzie w stanie analizować sekwencje ruchów w grze "trzy kubki". Uczestnicy mają za zadanie określić końcową pozycję kubków po serii ruchów, korzystając z analizy statycznych obrazów z każdej klatki nagrania.

In [1]:
######################### NIE ZMIENIAJ TEJ KOMÓRKI PODCZAS WYSYŁANIA ##########################

# Poniższe funkcje ułatwiają pracę z dostarczonymi danymi
# W kolejnych komórkach zobaczysz przykłady ich użycia
from utils.utils import get_level_info, get_video_data, display_video, download_and_replace_data

FINAL_EVALUATION_MODE = False
# W czasie sprawdzania Twojego rozwiązania, zmienimy tę wartość na True
# Wartość tej flagi M U S I zostać ustawiona na False w rozwiązaniu, które nam nadeślesz!
# if not FINAL_EVALUATION_MODE:
#     images, coordinates, _, path_to_images = get_video_data(level=3,video_id=0,dataset="example")
#     display_video(images,rescale=0.7,FINAL_EVALUATION_MODE=FINAL_EVALUATION_MODE)

## Zadanie 3: Zbuduj rozwiązanie od zera

W tym zadaniu podejdziesz do problemu identyfikacji obiektów na filmach wideo od podstaw. Poprzednie zadania wymagały użycia gotowych informacji o lokalizacji obiektów w klatkach. Tym razem będziesz musiał stworzyć algorytm, który będzie operować bezpośrednio na nieoznaczonych obrazach, co pozwoli na pełniejsze zrozumienie i opracowanie własnego systemu detekcji obiektów.

Napisz algorytm, który poradzi sobie ze zbiorem danych `level_3` bez podanych prostokątów ograniczających.

## Pliki zgłoszeniowe
Tylko ten notebook zawierający **kod** oraz **krótki raport** opisujący Twoje rozwiązanie (do 300 słów). Miejsce na raport znajdziesz na końcu tego notebooka.

## Ograniczenia
- Twoja funkcja powinna zwracać predykcje w maksymalnie 5 minut używając Google Colab bez GPU.

## Uwagi i wskazówki
- Testuj swoje rozwiązanie na zbiorze plików wideo `level_3`.
- **Skuteczność modelu**: przetestuj skuteczność modelu na zbiorze walidacyjnym używając dostarczonej przez nas funkcji **submission_script**, umieść ten wynik w raporcie.

## Ewaluacja
Pamiętaj, że podczas sprawdzania flaga `FINAL_EVALUATION_MODE` zostanie ustawiona na `True`. Za pomocą skryptu `validation_script.py` możesz upewnić się, że Twoje rozwiązanie zostanie prawidłowo wykonane na naszych serwerach oceniających.

Za to podzadanie możesz zdobyć pomiędzy 0 i 0.5 punktów. Zdobędziesz 0 punktów jeśli Twoje accuracy na zbiorze testowym będzie poniżej 30%. Jeśli będzie równe 100%, otrzymasz 0.5 punktu. Pomiędzy tymi wartościami, wynik rośnie liniowo z wartością metryki.

# Kod startowy

In [2]:
# Poniższe biblioteki są wystarczające do wykonania wszystkich zadań
# Jeśli jednak chcesz użyć innych, sprawdź czy są dostępne na serwerze (requirements.txt)
import numpy as np
import os
import matplotlib.pyplot as plt
import torch
import IPython.display
import json
import PIL
import sklearn as sk
import gdown
import os

In [5]:
# if not FINAL_EVALUATION_MODE:
#     # funkcja pomocnicza do ładowania danych
#     images, _, _, _ = get_video_data(level=3,video_id=0,dataset="example")

#     with open(os.path.join(os.getcwd(),'example_tracks','tracks_3_0.json'), 'r') as f:
#         tracks = json.load(f)

#     for key in tracks.keys():
#         tracks[key] = [tuple(el) for el in tracks[key]]

#     # funkcja pomocnicza do wyświetlania danych
#     display_video(images,
#                     tracks=tracks,
#                     rescale=0.7,
#                     FINAL_EVALUATION_MODE=FINAL_EVALUATION_MODE)

In [11]:
# Pobieranie danych do podzadań 1, 2 i 3 (około ~646Mb), skrypt będzie wykonywał się parę minut
# Wystarczy, że pobierzesz dane tylko raz. Na serwerze sprawdzającym dane będą już pobrane,
# struktura plików będzie identyczna jak tutaj
if not FINAL_EVALUATION_MODE:
    download_and_replace_data()


FileURLRetrievalError: Failed to retrieve file url:

	Too many users have viewed or downloaded this file recently. Please
	try accessing the file again later. If the file you are trying to
	access is particularly large or is shared with many people, it may
	take up to 24 hours to be able to view or download the file. If you
	still can't access a file after 24 hours, contact your domain
	administrator.

You may still be able to access the file from the browser:

	https://drive.google.com/uc?id=1gEV52fT3luVkTU_Qf-aTd2hWj-7rYCpQ

but Gdown can't. Please check connections and permissions.

In [7]:
######################### NIE ZMIENIAJ TEJ KOMÓRKI ##########################

# funkcja pomocnicza do testowania algorytmu
# Zwróć uwagę na to że funkcja ta działa inaczej niż w poprzednich podzadaniach
# algorytm przyjmuje jako argument listę obrazków, a nie koordynaty
def submission_script(algorithm,level,verbose=False,dataset="valid"):
    num_videos, _ = get_level_info(level=level,dataset=dataset)
    correct = []
    exception_messages = set()
    for video_number in range(num_videos):
        images, _, target, _ = get_video_data(level=level,video_id=video_number,dataset=dataset)
        # try:
        prediction = algorithm(images)
        if tuple(target) == tuple(prediction):
            correct.append(1)
        else:
            correct.append(0)
        if verbose:
            print(f"Video: animation_{str(video_number).zfill(4)}")
            print(f"Prediction: {prediction}")
            print(f"Target:     {target}")
            print(f"Score: {tuple(target) == tuple(prediction)}", end='\n\n')
        # except Exception as e:
        #     correct.append(0)
        #     exception_messages.add(str(e))
    if verbose:
        print(f"Accuracy: {np.mean(correct)}")
        print(f"Correctness: {correct}")
    return np.sum(correct) / num_videos, correct, exception_messages

# Twoje rozwiązanie

In [8]:
def your_algorithm_task_3(images): # nie zmieniaj nazwy funkcji
    def your_algorithm_task_2(coordinates): # nie zmieniaj nazwy funkcji
        def get_order(frame):
            frame = np.array(frame)

            if frame[0,0] >= frame[1,0] and frame[1,0] >= frame[2,0]:
                return [2,1,0]
            elif frame[1,0] >= frame[0,0] and frame[0,0] >= frame[2,0]:
                return [1,2,0]
            elif frame[0,0] >= frame[2,0] and frame[2,0] >= frame[1,0]:
                return [2,0,1]
            elif frame[1,0] >= frame[2,0] and frame[2,0] >= frame[0,0]:
                return [0,2,1]
            elif frame[2,0] >= frame[0,0] and frame[0,0] >= frame[1,0]:
                return [1,0,2]
            elif frame[2,0] >= frame[1,0] and frame[1,0] >= frame[0,0]:
                return [0,1,2]

        def distance(obj1, obj2):
            a = (((obj1[0]-obj2[0])**2) + ((obj1[1]-obj2[1])**2))**0.5

            return a

        def find_missing(table):
            missing = []
            if 0 not in table:
                missing.append(0)
            if 1 not in table:
                missing.append(1)
            if 2 not in table:
                missing.append(2)

            return missing

        def is_good_distance(arr):
            arr = sorted(arr)

            if(arr[0] == np.inf):
                return False
            if(arr[1] == np.inf):
                return True

            if arr[1] - arr[0] > -1:
                return True
            return False

        def get_distance(arr):
            arr = sorted(arr)

            if(arr[0] == np.inf):
                return -1
            if(arr[1] == np.inf):
                return np.inf

            return arr[1] - arr[0]

        def move(frame, prev_frame, order, save):

            if len(frame) == 0:
                return np.full(3,-1)

            output = np.full(3,-1)
            distances_matrix = np.full((3,3), np.inf)

            for i in range(len(frame)):
                for j in range(len(prev_frame)):
                    distances_matrix[i,j] = distance(frame[i], prev_frame[j])

            distances_in_row = np.array([get_distance(distances_matrix[i]) for i in range(len(distances_matrix))])
            while True:
                row = np.argmax(distances_in_row)
                distances_in_row[row] = -1

                if row == -1:
                    break

                flat_min = np.argmin(distances_matrix[row])
                idx = row,flat_min

                if(distances_matrix[idx] == np.inf):
                    break

                if(distances_matrix[idx] > (save[order[idx[1]]] + 1) * 50):
                    distances_matrix[idx[0]] = np.inf


                if(is_good_distance(distances_matrix[idx[0]])):
                    output[idx[0]] = order[idx[1]]
                    distances_matrix[:,idx[1]] = np.inf

                distances_matrix[idx[0],:] = np.inf

            return output

        def calc_vector(box, prev_box, v2):
            v1 = np.array([box[0] - prev_box[0], box[1] - prev_box[1]])

            sr = (v1 + np.array(v2))/2

            v1 = resize_vector(v1, vector_len(sr))

            if(vector_len(v1) > 50):
                v1 = np.array([0,0])

            return v1

        def add_vector(center, vector):
            return [center[0] + vector[0], center[1] + vector[1]]

        def vector_len(v1):
            return np.array((v1[0]**2 + (v1[1]**2))**0.5)

        def resize_vector(v1, n):
            if n==0:
                return v1/2
            if vector_len(v1) == 0:
                return np.array([0,0])

            epsilon = vector_len(v1)/n
            v1 = v1/epsilon
            return v1

        starting_order = get_order(coordinates[0])
        current_order = starting_order
        prev_frame = coordinates[0]
        vectors = np.array([[0,0],[0,0],[0,0]], dtype=np.double)

        last_save = np.array([0,0,0])

        for frame in range(len(coordinates)):

            centers = coordinates[frame]

            prev_order = current_order
            current_order = move(centers, prev_frame, current_order, last_save)

            new_frame = []

            missing = find_missing(current_order)
            missing_id = 0

            for i,el in enumerate(current_order):

                if el != -1:
                    new_frame.append(centers[i])

                    vectors[el] = calc_vector(new_frame[i], prev_frame[np.where(prev_order == el)[0][0]], vectors[el])
                    last_save[el] = 0
                else:
                    miss_id = np.where(prev_order == missing[missing_id])[0][0]
                    missing_id += 1

                    current_order[i] = prev_order[miss_id]
                    new_frame.append(add_vector(prev_frame[miss_id], vectors[prev_order[miss_id]]))
                    last_save[prev_order[miss_id]] += 1

            prev_frame = np.array(new_frame)



        final_order = get_order(prev_frame)

        end = np.zeros(3)
        for i in range(len(final_order)):
            end[final_order[i]] = current_order[i]

        return list(end.astype(np.int16))

    def get_distance(p1, p2):
        x1, y1 = p1
        x2, y2 = p2

        plain_dist = ((x2 - x1)**2 + (y2 - y1)**2)**2

        avg_y = 1
        if y1 and y2 != 0:
            avg_y = (y1+y2)/2

        include_perspective = plain_dist * ((100 / avg_y)/1)

        return include_perspective

    def find_three_farthest_points(points):

        distances = []
        for i in range(len(points)):
            for j in range(i+1, len(points)):
                distance = get_distance(points[i],points[j])
                distances.append((distance, i, j))

        distances.sort(reverse=True)

        result = []
        used = set()

        result.append(points[distances[0][1]])
        result.append(points[distances[0][2]])
        used.add(distances[0][1])
        used.add(distances[0][2])

        distances2 = []

        for i in range(len(points)):
            if i not in used:
                distances2.append((min(get_distance(result[0],points[i]), get_distance(result[1],points[i])), i))
        distances2.sort(reverse=True)

        result.append(points[distances2[0][1]])

        return result

    def is_cup(fi, j, i, val=50,split=12):
        if fi[j][i] < val and fi[j-split][i] < val and fi[j][i-split] < val and fi[j][i+split]< val and fi[j+split][i]< val:
            return True
        return False

    coordinates = []

    for img in images:
        fi = np.array(img)[:,:,2]

        coords = []

        split = 15

        strt = 6

        for i in range(split+strt, fi.shape[0]-split-1, split):
            for j in range(split+strt, fi.shape[1]-split-1, split):
                if is_cup(fi,i,j):
                    coords.append((j,i))

        if len(coords) != 0:
            coordinates.append(find_three_farthest_points(coords))

    wyn = your_algorithm_task_2(coordinates=coordinates)
    return wyn

In [9]:
# imgs, coordinates, _, _ = get_video_data(level=3, video_id=9, dataset="valid")

# wynik, kp = your_algorithm_task_3(imgs)

# print(wynik)

# display_video(imgs,
#               first_preds=kp[0],
#               second_preds=kp[1],
#               third_preds=kp[2],
#                 rescale=0.7,
#                 FINAL_EVALUATION_MODE=FINAL_EVALUATION_MODE)

In [12]:
if not FINAL_EVALUATION_MODE:
    # Sprawdź jak działa Twój algorytm
    accuracy, correctness, _ = submission_script(
        algorithm=your_algorithm_task_3,
        level=3,
        verbose=True,
        dataset="valid")

FileNotFoundError: [Errno 2] No such file or directory: '/content/valid_data/level_3/images'

In [13]:
# zapisz swój raport do zmiennej poniżej, abyśmy mogli go później automatycznie odczytać sprawdzaczką
raport_3 = \
"""
Raport z zadania:

Generalnie sposób działania jest taki sam jak w poziomie 2, jednak teraz napisałem własny algorytm wykrywania koordynatów (wykrywa on środki boxów, a nie wierzchołki).
Obrazki zamieniam na np.array kanału niebieskiego, bo na nim doskonale wyróżniają się kubeczki i na takich obrazka operuję.
Następnie przechodzę przez co 15 piksel od lewej do prawej i co 15 od góry do dołu i sprawdzam czy ten piksel oraz piksel 15 pikseli nad nim, pod nim, po lewej i po prawej
Mają wartość mniejszą niż 50 (funkcja 'is_cup') czyli czy znajdują się wewnątrz któregoś kubeczka i jeśli tak do zapisuję do tablicy. Później z tablicy wybieram trzy
najbardziej oddalone od siebie punkty i uznaję je za współrzędne kubeczków. Zapisuję to wszystko do zmeinnej 'coordinates' i następnie przekazuję ją funkcji z
popunktu drugiego (działa dokładnie tak samo, ze zmianą kilku drobnych parametrów tak by działało dla tablicy, a nie słownika) i otrzymuję wynik.

Zachęcam do rzucenia okiem na kod, jeśli coś byłoby niejasne i dziękuję za przeczytanie mojego rozwiązania.
"""